Exchange
========
PaperExchange
-------------
The PaperExchange is for simulation back-testing

In [1]:
import pandas as pd
import data as datalib

from data import data_manager, market_data_manager
from tomahawk import exchange

In [2]:
# Initialize the PaperExchange
pe = exchange.PaperExchange()

In [3]:
# Receive and order. The PaperExchange does not take in the order object but just the relevant order details
order_id = pe.receive_order('stock', 'TEST', 'sell', 30, 'LIMIT', price=10.10)
pe.open_orders_df.show()

             index  product_type    symbol    state    buy_sell      quantity  order_type    fill_quantity    fill_price      price            order_id  replaces
------------------  --------------  --------  -------  ----------  ----------  ------------  ---------------  ------------  -------  ------------------  -----------------------------------------------
170502084518000001  stock           TEST      LIVE     sell                30  LIMIT                                           10.1  170502084518000001  [Replace(quantity=30, details={'price': 10.1})]


In [4]:
# Now fill the order and look at closed orders. This should not be done directly and is for demo only
pe.fill_order(pe.get_order(order_id), 30, pd.Timestamp('2010-01-04 10:01:00', tz='America/New_York'))
pe.closed_orders_df.show()

             index  product_type    symbol    state    buy_sell      quantity  order_type      fill_quantity    fill_price    price            order_id  replaces                                         fills                                                                                                                           close_bar_timestamp
------------------  --------------  --------  -------  ----------  ----------  ------------  ---------------  ------------  -------  ------------------  -----------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------  -------------------------
170502084518000001  stock           TEST      FILLED   sell                30  LIMIT                      30          10.1     10.1  170502084518000001  [Replace(quantity=30, details={'price': 10.1})]  [Fill(id=170502084518000001, timestamp=Timestamp('2010-01-04 10:01:00-0500', tz='America/New

In [5]:
# The proper way is to use process_orders() to run a full loop on all symbols and outstanding orders for the current bar
# This requires setting up all the pre-requesite objects

csvdf = datalib.CsvDataFeed('c:/rmbaries/git/pifq/src/puma/data/tests/inst/csv_data_feed')
lmdm = data_manager.LiveDataManager(csvdf, **credentials('test', 'localhost', 'db_'))
mdm = market_data_manager.MarketDataManager(None, lmdm)
mdm.add_symbols('stock', 'test.sym.3', '1min')

In [6]:
# setup the PaperExchange and receive some orders

pe = exchange.PaperExchange()
id_1 = pe.receive_order('stock', 'test.sym.3', 'sell', 30, 'LIMIT', price=10.10)
id_2 = pe.receive_order('stock', 'test.sym.3', 'buy', 130, 'LIMIT', price=10.10)

In [7]:
# Now set the bartime on the MarketDataManager, update the data and process the orders

mdm.bartime = '2010-01-01 09:31:00'
mdm.update('stock', '1min')
pe.process_orders(mdm)

In [8]:
# the remaining open orders
print(pe.open_orders_df)

             index  product_type    symbol      state    buy_sell      quantity  order_type    fill_quantity    fill_price      price            order_id  replaces
------------------  --------------  ----------  -------  ----------  ----------  ------------  ---------------  ------------  -------  ------------------  ------------------------------------------------
170502084519000002  stock           test.sym.3  LIVE     buy                130  LIMIT                                           10.1  170502084519000002  [Replace(quantity=130, details={'price': 10.1})]


In [9]:
# the closed orders
print(pe.closed_orders_df)

             index  product_type    symbol      state    buy_sell      quantity  order_type      fill_quantity    fill_price    price            order_id  replaces                                         fills                                                                                                                           close_bar_timestamp
------------------  --------------  ----------  -------  ----------  ----------  ------------  ---------------  ------------  -------  ------------------  -----------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------  -------------------------
170502084519000001  stock           test.sym.3  FILLED   sell                30  LIMIT                      30          10.1     10.1  170502084519000001  [Replace(quantity=30, details={'price': 10.1})]  [Fill(id=170502084519000001, timestamp=Timestamp('2010-01-01 09:31:00-0500', tz='Ameri

Parameters
----------
PaperExchange() can be initialized with overrides of the paramaters

In [10]:
paper_ex = exchange.PaperExchange(parameters={'fill_multiplier': 0.75})

paper_ex.parameters

{'fill_multiplier': 0.75}